# Load

In [ ]:
import numpy as np
import pandas as pd
import polars as pl
import os

import matplotlib.pyplot as plt
import seaborn as sns

import pathlib
from pathlib import Path

directory_raw = str(Path.cwd().parent) + '\\data_raw\\economy\\'

directory_processed = str(Path.cwd().parent) + '\\data\\economy\\'

# Functions

In [ ]:
def read_data(file_name):
    # data = pl.from_pandas(pd.read_csv(directory_raw+'/'+file_name))
    data = pl.from_pandas(pd.read_csv(directory_raw + file_name))
    return data

def save_data(data, file_name):
    data.write_csv(directory_processed + file_name + ".csv")
    print(f"{file_name} has been saved.")

def get_country_income_data(file_name):
    data = (
        pd.read_csv(
            directory_raw + file_name, 
            sep=';', 
            names=['category', 'year', 'share'],
        )
        .assign(country=f"{file_name.replace('income_','').replace('.csv','')}")
        )
    return data

# income_total

In [ ]:
income_total = pl.from_pandas(
    pd.read_csv(directory_raw + "personal_income_fred.csv")
    .assign(
        year = pd.to_datetime(pd.read_csv(directory_raw + "personal_income_fred.csv")['year']).dt.year,
    )
).with_columns(income=pl.col('income_in_billions') * 1000000000)
save_data(data=income_total, file_name="income_total")
income_total

# Dropbox/Breathe/data_raw/personal_income_fred.csv

# population

In [ ]:
population = pl.from_pandas(
    pd.read_csv(directory_raw + "population_fred.csv")
    .assign(
        year = pd.to_datetime(pd.read_csv(directory_raw + "population_fred.csv")['date']).dt.year,
    )
)
save_data(data=population, file_name="population")
population

# workers_ratio

In [ ]:
workers_ratio = pl.from_pandas(
    pd.read_csv(directory_raw + "workers_ratio_fred.csv")
    .assign(
        year = pd.to_datetime(pd.read_csv(directory_raw + "workers_ratio_fred.csv")['date']).dt.year,
    )
).with_columns(ratio = pl.col('ratio')/100)
save_data(data=workers_ratio, file_name="workers_ratio")
workers_ratio

# shares_wid

In [ ]:
files = [
    'income_switzerland.csv',
    'income_australia.csv',
    'income_france.csv',
    'income_canada.csv',
    'income_china.csv',
    'income_germany.csv',
    'income_italy.csv',
    'income_japan.csv',
    'income_new_zealand.csv',
    'income_norway.csv',
    'income_russia.csv',
    'income_uk.csv',
    'income_usa.csv',
]
data_temp = pd.DataFrame()
for f in files:
    data_temp = pd.concat([
        data_temp, 
        get_country_income_data(file_name=f)
    ])

shares_wid = (
    # bottom = bottom 50% percentile of income
    # middle = 51-99% percentile of income
    # top = 99-100% percentile of income
    pl.from_pandas(data_temp)
    .pivot(on='category', index=['country', 'year',], values='share')
    .drop_nulls()
    .select([
        'country', 'year',
        pl.col('p0p50').alias('share_bottom'),
        pl.col('p99p100').alias('share_top'),
    ])
    .with_columns(
        share_upper = pl.lit(1) - pl.col('share_top') - pl.col('share_bottom'), 
    )
    .sort(['country', 'year'])
)
shares_wid

In [ ]:
year_max = shares_wid.select(pl.max('year')).to_numpy().flatten()[0]
income_total_year_max = (
    income_total
    .filter(pl.col('year')==year_max)
    .select('income')
    .to_numpy().flatten()[0]
)
workers_ratio_year_max = (
    workers_ratio
    .filter(pl.col('year')==year_max)
    .select('ratio')
    .to_numpy().flatten()[0]
)
workers_year_max = (
    population
    .filter(pl.col('year')==year_max)
    .select('population')
    .to_numpy().flatten()[0]
    * workers_ratio_year_max
)
print(f"year_max: {year_max}")
print(f"income_total_year_max: {int(income_total_year_max):,}")
print(f"workers_ratio_year_max: {workers_ratio_year_max:,.2}")
print(f"workers_year_max: {int(workers_year_max):,}")

In [ ]:
"""
Brundage, Vernon - U.S. Bureau of Labor Statistics. USUAL WEEKLY EARNINGS of 
WAGE and SALARY WORKERS THIRD QUARTER 2016. 2024. www.bls.gov/news.release/pdf/wkyeng.pdf
"""
# $1,145 full-time median earnings 2023 Q4, n_workers 120,797 in thousands
# $362 part-time median earnings 2023 Q4, n_workers 24,010 in thousands
income_yearly_part_time = 362 * 52
income_yearly_full_time = 1145 * 52

print(f"part_time yearly: {income_yearly_part_time:,.0f}")
print(f"full_time yearly: {income_yearly_full_time:,.0f}")

print(f"part_time pct of income: {income_yearly_part_time/(income_yearly_part_time+income_yearly_full_time):.1%}")

full_time_pct_of_income = income_yearly_full_time/(income_yearly_part_time+income_yearly_full_time)
print(f"full_time pct of income: {full_time_pct_of_income:.1%}")

In [ ]:
"""
Brundage, Vernon - U.S. Bureau of Labor Statistics. USUAL WEEKLY EARNINGS of 
WAGE and SALARY WORKERS THIRD QUARTER 2016. 2024. www.bls.gov/news.release/pdf/wkyeng.pdf
"""
# $1,145 full-time median earnings 2023 Q4, n_workers 120,797 in thousands
# $362 part-time median earnings 2023 Q4, n_workers 24,010 in thousands
n_workers_full_time = 120797 * 1000
n_workers_part_time = 24010 * 1000

print(f"n_workers_part_time: {n_workers_part_time:,.0f}")
print(f"n_workers_full_time: {n_workers_full_time:,.0f}")
print(f"n_workers total: {n_workers_full_time + n_workers_part_time:,.0f}")

n_workers_part_time_pct = n_workers_part_time/(n_workers_part_time+n_workers_full_time)
print(f"part_time pct: {n_workers_part_time_pct:.1%}")
print(f"full_time pct: {n_workers_full_time/(n_workers_part_time+n_workers_full_time):.1%}")

part_time_pct_of_bottom_half = n_workers_part_time/((n_workers_part_time+n_workers_full_time)/2)
print(f"part_time n_workers pct of bottom half: {part_time_pct_of_bottom_half:.1%}")

full_time_pct_of_bottom_half = 1 - part_time_pct_of_bottom_half
print(f"full_time n_workers pct of bottom half: {full_time_pct_of_bottom_half:.1%}")

In [ ]:
# using fred's population counts to stay consistant with the sources
# of other data
n_workers_fred_part_time = workers_year_max * n_workers_part_time_pct
n_workers_fred_full_time = workers_year_max - n_workers_fred_part_time
print(f"n_workers_fred_part_time: {n_workers_fred_part_time:,.0f}")
print(f"n_workers_fred_full_time: {n_workers_fred_full_time:,.0f}")

print("\n")
n_workers_full_time_top_1 = workers_year_max * 0.01
n_workers_full_time_bottom_50 = (workers_year_max * 0.50) - n_workers_fred_part_time
print(f"n_workers_fred top 1% just full_time: {n_workers_full_time_top_1:,.0f}")
print(f"n_workers_fred bottom 50% just full_time: {n_workers_full_time_bottom_50:,.0f}")

save_data(
    data=pl.from_pandas(pd.DataFrame({
        "group": ["top_1", "bottom_50"],
        "n_workers_full_time": [n_workers_full_time_top_1, n_workers_full_time_bottom_50],
        })),
    file_name='n_workers_full_time',
)

In [ ]:
shares_wid = (
    shares_wid
    .with_columns(
        # mean numbers for full-time employment
        income_mean_bottom = (
            # income
            # first get the amount of income in the bottom_50%
            # then remove the part-time income from that pool using what we know
            # about the annual yearly income of part-time employment and the 
            # count of fred population that work part-time
            ((income_total_year_max * pl.col('share_bottom')) - (n_workers_fred_part_time * income_yearly_part_time) )
            / 
            # n_workers
            # the first 50% of income earniners will include the part-time group (assumption)
            # first calculate the count of the bottom_50%, then subtract the number calculated
            # above, n_workers_fred_part_time from this group
            ((workers_year_max * 0.50) - n_workers_fred_part_time)         
        ).cast(int),
        
        income_mean_upper = ((income_total_year_max * pl.col('share_upper')) / (workers_year_max * 0.49)).cast(int),
        income_mean_top = ((income_total_year_max * pl.col('share_top')) / (workers_year_max * 0.01)).cast(int),      
    )
    .with_columns(
        multiple = (pl.col('income_mean_top') / pl.col('income_mean_bottom') *10).cast(int) / 10,
        income_mean_gap = pl.col('income_mean_top') - pl.col('income_mean_bottom'),
    )
)
save_data(data=shares_wid, file_name="shares_wid")
shares_wid

## yearly income calculations

In [ ]:
data = (
    shares_wid
    .filter(pl.col('country') == 'usa')
    .filter(pl.col('year') == year_max)
    .select([
        'country', 'year',
        pl.col('share_top').alias('richest_person'),
        pl.col('share_bottom').alias('50_poorest'),
    ])
    .unpivot(
        index=['country', 'year'],
        on=[
            '50_poorest',
            'richest_person',
        ],
        variable_name='quantile',
        value_name='share',
    )
    .filter(pl.col('quantile').is_in(['50_poorest', 'richest_person']))
)

bottom_share = (
    data
    .filter(pl.col('quantile')=="50_poorest")
    .select('share')
    .to_numpy().flatten()[0]
)
income_mean_bottom = (income_total_year_max * bottom_share) / (workers_year_max * 0.50)
top_share = (
    data
    .filter(pl.col('quantile') == "richest_person")
    .select('share')
    .to_numpy().flatten()[0]
)
income_mean_top = (income_total_year_max * top_share) / (workers_year_max * 0.01)
multiple = int(income_mean_top/income_mean_bottom)

print(f"bottom_share: {bottom_share:,.2}")
print(f"income_mean_bottom: {int(income_mean_bottom):,}")
print(f"top_share: {top_share:,.2}")
print(f"income_mean_top: {int(income_mean_top):,}")

# Taxes

In [ ]:
pos_idx = 0
neg_idx = 0
change = 3

# (pos_idx+=1) if (change>0) else (pos_idx+=)0
pos_idx = (pos_idx + 1) if (change>0) else pos_idx
neg_idx = (neg_idx + 1) if (change<0) else neg_idx

In [ ]:
tax = (
    read_data(file_name='tax_rates_income.csv')
    .select([
        pl.col('year').cast(int),
        pl.col('rate').cast(float),
        pl.col('income').cast(int),
    ])
    .with_columns(
        rate_bottom_bracket=pl.min('rate').over(partition_by=['year']),
        rate_median=pl.median('rate').over(partition_by=['year']),
        rate_top_bracket=pl.max('rate').over(partition_by=['year']),
    )
    .with_columns(
        change_top_bracket = pl.col('rate_top_bracket') - pl.col('rate_top_bracket').shift(1)
    )
    .select([
        'year',
        'rate_bottom_bracket', 'rate_median', 'rate_top_bracket', 'change_top_bracket',
    ])
    .unique(subset=['year'])
    .sort(['year'])
)
save_data(data=tax, file_name='tax')
tax

# taxes corporate

In [ ]:
tax_corporate = (
    read_data(file_name='tax_corporate.csv')
    .with_columns(
        pl.col('year').str.extract(r'(\d+)-').alias('first'),
        pl.col('year').str.extract(r'-(\d+)').alias('last'),
    )
    .with_columns(
        pl.sql_expr("""
            CASE
                WHEN (first is NULL) THEN year
                ELSE first
            END
        """).alias('first')
    )
    .with_columns(
        pl.sql_expr("""
            CASE
                WHEN (last is NULL) THEN first
                ELSE last
            END
        """).alias('last')
    )
    .select([
        'first', 'last', 'brackets', 'rate',
    ])
    .sort(['first'])
)
tax_corporate

# shares_wid_full_distribution

In [ ]:
file_name="income_share_distribution_before_tax_world_wid.csv"

shares_wid_full_distribution = (
    read_data(file_name=file_name)
    .select([
        pl.col('Entity').alias('country'),
        pl.col('Year').alias('year'),
        pl.col('Top 1% - Share (Pretax) (Estimated)').alias('share_top') / 100,
        pl.col('Between 90th and 99th percentiles - Share (Pretax) (Estimated)').alias('upper_top_9') / 100,
        pl.col('Middle 40% - Share (Pretax) (Estimated)').alias('upper_middle_40') / 100,
        pl.col('Bottom 50% - Share (Pretax) (Estimated)').alias('share_bottom') / 100,
    ])
    .with_columns(
        country = pl.sql_expr("""
            CASE
                WHEN country='United States' THEN 'usa'
                WHEN country='United Kingdom' THEN 'uk'
                WHEN country='New Zealand' THEN 'new_zealand'
                ELSE country
            END
        """).str.to_lowercase(),
        gap = pl.col('share_top') - pl.col('share_bottom'),
    )
)
countries = shares_wid_full_distribution.select('country').unique()['country'].to_numpy().flatten().tolist()
[col for col in countries if "United" in col] # United States

# shares

save_data(
    data=shares_wid_full_distribution, 
    file_name="shares_wid_full_distribution",
)
shares_wid_full_distribution

# EDA

In [ ]:
shares_data = shares_wid

In [ ]:
cols = ['income_mean_bottom', 'income_mean_upper', 'income_mean_top',]
# cols = ['share_bottom', 'share_upper', 'share_top',]
data = (
    shares_data
    .filter(pl.col('country') == 'usa')
    .filter(pl.col('year') == year_max)
    .select([
        'country', 'year',
        'income_mean_bottom', 'income_mean_upper', 'income_mean_top',
    ])
    .unpivot(
        index=['country', 'year',],
        on=cols,
        variable_name='group',
        value_name='value',
    )
    .with_columns(
        bin_bottom=pl.sql_expr(f"""
            CASE
                WHEN group='{[col for col in cols if 'bottom' in col][0]}' THEN 0
                WHEN group='{[col for col in cols if 'upper' in col][0]}' THEN 51
                WHEN group='{[col for col in cols if 'top' in col][0]}' THEN 99
                ELSE NULL
            END
        """),
        bin_top=pl.sql_expr(f"""
            CASE 
                WHEN group='{[col for col in cols if 'bottom' in col][0]}' THEN 50
                WHEN group='{[col for col in cols if 'upper' in col][0]}' THEN 98
                WHEN group='{[col for col in cols if 'top' in col][0]}' THEN 100
                ELSE NULL
            END
        """),
    )
    .filter(pl.col('group').is_in(cols))
    .sort(['bin_bottom'])
)
data.shape[0]
data

In [ ]:
def get_income_mean(group, data):
    income_mean = (
        data
        .filter(pl.col('group') == group)
        .select('value')
        .to_numpy().flatten()[0]
    )
    return income_mean
    
income_mean_bottom = get_income_mean(group="income_mean_bottom", data=data)
income_mean_upper = get_income_mean(group="income_mean_upper", data=data)
income_mean_top = get_income_mean(group="income_mean_top", data=data)

text_income_mean_bottom = [
    f"<b>${income_mean_bottom:,.0f}k</b>/yr average"
]
text_income_mean_upper = [
    f"<b>${income_mean_upper:,.0f}k</b>/yr average"
]
text_income_mean_top = [
    f"<b>${income_mean_top:,.0f} million</b>/yr average <br>{multiple:}x mainstreet"
]

text_income_mean_bottom

In [ ]:
usa = shares_data.filter(pl.col('country')=='usa')

plt.plot(usa['year'], usa['share_bottom'] + usa['share_top'])
plt.title("What is the combined share of top & bottom")

In [ ]:
f, ax = plt.subplots(1,1, figsize=[22,8])
sns.lineplot(data=shares_data, x='year', y='income_mean_bottom', hue='country', ax=ax)

last = shares_data.filter(pl.col('year')==year_max)
for country in last['country']:
    data = last.filter(pl.col('country')==country)
    plt.text(x=data['year'].to_numpy()[0], y=data['income_mean_bottom'].to_numpy()[0], s=data['country'].to_numpy()[0])

plt.title("What is main street income over time?", fontsize=28)

In [ ]:
f, ax = plt.subplots(1,1, figsize=[22,8])
sns.lineplot(data=shares_data, x='year', y='multiple', hue='country', ax=ax)

last = shares_data.filter(pl.col('year')==year_max)
for country in last['country']:
    data = last.filter(pl.col('country')==country)
    plt.text(x=data['year'].to_numpy()[0], y=data['multiple'].to_numpy()[0], s=data['country'].to_numpy()[0])

plt.title("What is the multiple over time?", fontsize=28)